<a href="https://colab.research.google.com/github/pouvelenc/DD2365-aflum/blob/main/project_reynolds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **DD2365 - Project**
## **Adaptive algorithm for NS with slip-with-friction BCs**
**Povel Kann**

## **Environment setup**

In [ ]:
# Load neccessary modules.
from google.colab import files

import numpy as np
import time

# Install FEniCS (this may take a long time)
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install-release-real.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin

from dolfin import *; from mshr import *

import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt

from matplotlib.animation import FuncAnimation

from matplotlib.animation import FFMpegWriter

import matplotlib as mpl

from IPython.display import HTML

import logging

logging.getLogger('FFC').setLevel(logging.CRITICAL)
logging.getLogger('UFL').setLevel(logging.CRITICAL)
logging.getLogger('DOLFIN').setLevel(logging.CRITICAL)

--2025-05-28 17:51:22--  https://fem-on-colab.github.io/releases/fenics-install-release-real.sh
Resolving fem-on-colab.github.io (fem-on-colab.github.io)... 185.199.110.153, 185.199.111.153, 185.199.108.153, ...
Connecting to fem-on-colab.github.io (fem-on-colab.github.io)|185.199.110.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4180 (4.1K) [application/x-sh]
Saving to: ‘/tmp/fenics-install.sh’

/tmp/fenics-install 100%[===================>]   4.08K  --.-KB/s    in 0.004s  

2025-05-28 17:51:22 (1011 KB/s) - ‘/tmp/fenics-install.sh’ saved [4180/4180]

+ INSTALL_PREFIX=/usr/local
++ echo /usr/local
++ awk -F/ '{print NF-1}'
+ INSTALL_PREFIX_DEPTH=2
+ PROJECT_NAME=fem-on-colab
+ SHARE_PREFIX=/usr/local/share/fem-on-colab
+ FENICS_INSTALLED=/usr/local/share/fem-on-colab/fenics.installed
+ [[ ! -f /usr/local/share/fem-on-colab/fenics.installed ]]
+ PYBIND11_INSTALL_SCRIPT_PATH=https://github.com/fem-on-colab/fem-on-colab.github.io/raw/a51d633e/releases/pybi

### **Mesh**

In [ ]:
# define domain
length = 3
height = 3

# define circle
xc = 1.5
yc = 0.5*height
rc = 0.3

# define subdomains for boundary conditions
class Left(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[0], 0.0)

class Right(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[0], length)

class Lower(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], 0.0)

class Upper(SubDomain):
    def inside(self, x, on_boundary):
        return near(x[1], height)

class Skin_Friction(SubDomain):
    def inside(self, x, on_boundary):
        return (on_boundary and (not near(x[0], 0.0)) and (not near(x[0], length)) \
                and (not near(x[1], 0.0)) and (not near(x[1], height)))

left = Left()
right = Right()
lower = Lower()
upper = Upper()
skin_friction = Skin_Friction()

# generate mesh
resolution = 24
mesh = generate_mesh(Rectangle(Point(0.0,0.0), Point(length,height)) - Circle(Point(xc,yc),rc), resolution)

plt.figure()
plot(mesh)
plt.show()

## **Code**

#### **Mesh refinement**

In [ ]:
def solve_NS(mesh, beta, nu):

    # define mesh functions for boundary conditions
    boundaries = MeshFunction("size_t", mesh, mesh.topology().dim()-1)
    boundaries.set_all(0)
    left.mark(boundaries, 1)
    right.mark(boundaries, 2)
    lower.mark(boundaries, 3)
    upper.mark(boundaries, 4)
    skin_friction.mark(boundaries, 5)

    # finite element spaces (for velocity and pressure)
    V = VectorFunctionSpace(mesh, "Lagrange", 1)
    Q = FunctionSpace(mesh, "Lagrange", 1)

    # trial and test functions
    u = TrialFunction(V)
    p = TrialFunction(Q)
    v = TestFunction(V)
    q = TestFunction(Q)

    # boundary conditions
    class DirichletBoundaryLower(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[1], 0.0)

    class DirichletBoundaryUpper(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[1], height)

    class DirichletBoundaryLeft(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[0], 0.0)

    class DirichletBoundaryRight(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and near(x[0], length)

    class DirichletBoundaryObjects(SubDomain):
        def inside(self, x, on_boundary):
            return on_boundary and (not near(x[0], 0.0)) and (not near(x[0], length)) and (not near(x[1], 0.0)) and (not near(x[1], height))

    dbc_lower = DirichletBoundaryLower()
    dbc_upper = DirichletBoundaryUpper()
    dbc_left = DirichletBoundaryLeft()
    dbc_right = DirichletBoundaryRight()
    dbc_objects = DirichletBoundaryObjects()

    # time dependent and stationary inflow conditions
    #uin = Expression('4.0*x[1]*(1-x[1])', element = V.sub(0).ufl_element())
    #uin = Expression('1.0 + 1.0*fabs(sin(t))', element = V.sub(0).ufl_element(), t=0.0)
    bcu_in0 = DirichletBC(V.sub(0), uin, dbc_left)
    bcu_in1 = DirichletBC(V.sub(1), 0.0, dbc_left)
    bcu_upp0 = DirichletBC(V.sub(0), 0.0, dbc_upper)
    bcu_upp1 = DirichletBC(V.sub(1), 0.0, dbc_upper)
    bcu_low0 = DirichletBC(V.sub(0), 0.0, dbc_lower)
    bcu_low1 = DirichletBC(V.sub(1), 0.0, dbc_lower)
    bcu_obj0 = DirichletBC(V.sub(0), 0.0, dbc_objects)
    bcu_obj1 = DirichletBC(V.sub(1), 0.0, dbc_objects)

    pout = 0.0
    bcp1 = DirichletBC(Q, pout, dbc_right)

    #bcu = [bcu_in0, bcu_in1, bcu_upp0, bcu_upp1, bcu_low0, bcu_low1, bcu_obj0, bcu_obj1]
    #bcu = [bcu_in0, bcu_in1, bcu_upp1, bcu_low1, bcu_obj0, bcu_obj1]
    bcu = [bcu_in0, bcu_in1, bcu_upp1, bcu_low1]
    bcp = [bcp1]

    # measure for boundary integration
    ds = Measure('ds', domain=mesh, subdomain_data=boundaries)

    # iteration functions
    # (u0,p0) solution from previous time step
    # (u1,p1) linearized solution at present time step
    u0 = Function(V)
    u1 = Function(V)
    p0 = Function(Q)
    p1 = Function(Q)

    # parameters for nonlinear and lienar solvers
    num_nnlin_iter = 5
    prec = "amg" if has_krylov_solver_preconditioner("amg") else "default"


    # variational problem

    # stabilization parameters
    h = CellDiameter(mesh);
    u_mag = sqrt(dot(u1,u1))
    d1 = 4.0/sqrt((pow(1.0/dt,2.0) + pow(u_mag/h,2.0)))
    d2 = 2.0*h*u_mag

    # slip-with-friction boundary conditions, implemented as penalty terms
    e_n = FacetNormal(mesh)
    e_t = as_vector([e_n[1], -e_n[0]])
    C_alpha = 1.0e2
    alpha = C_alpha/h
    omega = 1.0

    # angular (tangential) velocity
    u_t = -1.0

    # smagorinsky model (simplified)
    C_t = 1.0e-2

    # mean velocities for trapozoidal time stepping
    um = 0.5*(u + u0)
    um1 = 0.5*(u1 + u0)

    # momentum variational equation on residual form
    Fu = inner((u - u0)/dt + grad(um)*um1, v)*dx \
        - p1*div(v)*dx \
        + nu*inner(grad(um), grad(v))*dx \
        + C_t*h*h*sqrt(inner(grad(um1), grad(um1)))*inner(grad(um), grad(v))*dx \
        + d1*inner((u - u0)/dt + grad(um)*um1 + grad(p1), grad(v)*um1)*dx \
        + d2*div(um)*div(v)*dx \
        + alpha*(inner(dot(um,e_n), dot(v,e_n)))*ds(5) \
        + beta*(inner(dot(um,e_t) - u_t, dot(v,e_t)))*ds(5)

    au = lhs(Fu)
    Lu = rhs(Fu)

    # continuity variational equation on residual form
    Fp = d1*inner((u1 - u0)/dt + grad(um1)*um1 + grad(p), grad(q))*dx + div(um1)*q*dx
    ap = lhs(Fp)
    Lp = rhs(Fp)


    # define the direction of the force to be computed
    phi_x_D = 1.0
    phi_y_D = 0.0
    phi_x_L = 0.0
    phi_y_L = 1.0

    psi_expression_D = Expression(("near(pow(x[0]-xc,2.0) + pow(x[1]-yc,2.0) - pow(rc,2.0), 0.0) ? phi_x : 0.","near(pow(x[0]-xc,2.0) + pow(x[1]-yc,2.0) - pow(rc,2.0), 0.0) ? phi_y : 0."), xc=xc, yc=yc, rc=rc, phi_x=phi_x_D, phi_y=phi_y_D, element = V.ufl_element())
    psi_D = interpolate(psi_expression_D, V)

    psi_expression_L = Expression(("near(pow(x[0]-xc,2.0) + pow(x[1]-yc,2.0) - pow(rc,2.0), 0.0) ? phi_x : 0.","near(pow(x[0]-xc,2.0) + pow(x[1]-yc,2.0) - pow(rc,2.0), 0.0) ? phi_y : 0."), xc=xc, yc=yc, rc=rc, phi_x=phi_x_L, phi_y=phi_y_L, element = V.ufl_element())
    psi_L = interpolate(psi_expression_L, V)

    Force_D = inner((u1 - u0)/dt + grad(um1)*um1, psi_D)*dx - p1*div(psi_D)*dx + nu*inner(grad(um1), grad(psi_D))*dx
    Force_L = inner((u1 - u0)/dt + grad(um1)*um1, psi_L)*dx - p1*div(psi_L)*dx + nu*inner(grad(um1), grad(psi_L))*dx

    # force normalization
    D = 2*rc
    normalization = -2.0/D

    # force computation data
    force_array_D = np.array(0.0)
    force_array_D = np.delete(force_array_D, 0)
    force_array_L = np.array(0.0)
    force_array_L = np.delete(force_array_L, 0)
    time = np.array(0.0)
    time = np.delete(time, 0)
    start_sample_time = 1.0

    us = []
    ps = []

    # time stepping
    T = 4
    t = dt
    while t < T + DOLFIN_EPS:

        # solve non-linear problem
        k = 0
        while k < num_nnlin_iter:

            # assemble momentum matrix and vector
            Au = assemble(au)
            bu = assemble(Lu)

            # compute velocity solution
            [bc.apply(Au, bu) for bc in bcu]
            [bc.apply(u1.vector()) for bc in bcu]
            solve(Au, u1.vector(), bu, "bicgstab", "default")

            # assemble continuity matrix and vector
            Ap = assemble(ap)
            bp = assemble(Lp)

            # compute pressure solution
            [bc.apply(Ap, bp) for bc in bcp]
            [bc.apply(p1.vector()) for bc in bcp]
            solve(Ap, p1.vector(), bp, "bicgstab", prec)

            # force
            F_D = assemble(Force_D)
            F_L = assemble(Force_L)
            if (t > start_sample_time):
              force_array_D = np.append(force_array_D, normalization*F_D)
              force_array_L = np.append(force_array_L, normalization*F_L)
              time = np.append(time, t)

            k += 1

        # update time step
        us.append(Function(V))
        ps.append(Function(Q))
        us[-1].assign(u1)
        ps[-1].assign(p1)

        u0.assign(u1)
        p0.assign(p1)
        t += dt

    primal_solutions = [(us[i], ps[i]) for i in range(len(us))]


    # plot solution
    plt.figure()
    plot(u1, title=f"velocity, beta={beta}, Re={Re}")
    plt.show()

    """
    plt.figure()
    plt.title(f"force, beta={beta}, Re={Re}")
    plt.plot(time, force_array_D, label="drag")
    plt.plot(time, force_array_L, label="lift")
    plt.legend()
    plt.show()
    """


    # finite element spaces (for adjoint velocity and pressure)
    VEa = VectorElement("CG", mesh.ufl_cell(), 2)
    QEa = FiniteElement("CG", mesh.ufl_cell(), 1)
    WEa = VEa * QEa

    Wa = FunctionSpace(mesh, WEa)

    # adjoint trial and test functions
    wa = Function(Wa)
    (phi, theta) = TrialFunctions(Wa)
    (va, qa) = TestFunctions(Wa)

    adjoint_bcs = [DirichletBC(Wa.sub(0), (0, 0), boundaries, 1),
                  DirichletBC(Wa.sub(0), (0, 0), boundaries, 2),
                  DirichletBC(Wa.sub(0), (0, 0), boundaries, 3),
                  DirichletBC(Wa.sub(0), (0, 0), boundaries, 4),
                  DirichletBC(Wa.sub(0), psi_L, boundaries, 5)]

    adjoint_solutions = []
    wa_n = Function(Wa)

    for t_step in reversed(range(len(us))):
        u_n = us[t_step]
        p_n = ps[t_step]

        phi_n, theta_n = split(wa_n)

        a_adjoint = (
            inner(phi, va)/dt * dx
          - inner(dot(u_n, grad(phi)), va)*dx
          + inner(dot(grad(u_n), phi), va)*dx
          + nu*inner(grad(va), grad(phi))*dx
          - inner(theta, div(va))*dx
          + inner(div(phi), qa)*dx
        )

        L_adjoint = inner(phi_n, va)/dt*dx

        solve(a_adjoint == L_adjoint, wa, bcs=adjoint_bcs)
        adjoint_solutions.insert(0, wa.copy(deepcopy=True))
        wa_n.assign(wa)

    phi_adjoint = wa.split()[0]

    """
    plt.figure()
    plot(phi_adjoint, title="adjoint velocity")
    plt.show()
    """

    return primal_solutions, adjoint_solutions, force_array_D, force_array_L, time


def compute_error_indicators(primal_sols, adjoint_sols, mesh, dt, nu, C_h, C_k):
    WDG  = FunctionSpace(mesh, "DG", 0)
    err_ind = Function(WDG)
    err_ind.vector()[:] = 0.0
    h = CellDiameter(mesh)

    # integrate over time
    for t_step in range(1, len(primal_sols)):
        u, p = primal_sols[t_step]
        u0, p0 = primal_sols[t_step - 1]

        phi, theta = adjoint_sols[t_step].split(deepcopy=True)
        phi0, theta0 = adjoint_sols[t_step - 1].split(deepcopy=True)

        R1 = (u - u0) / dt + dot(grad(u), u) + grad(p) - nu * div(grad(u))
        R2 = (nu / h) * inner(grad(u), grad(u))
        R3 = div(u)

        R1_norm = sqrt(inner(R1, R1))
        R2_norm = sqrt(inner(R2, R2))
        R3_norm = sqrt(inner(R3, R3))

        # time derivatives
        phi_dot = (phi - phi0) / dt
        phi_dot_norm = sqrt(inner(phi_dot, phi_dot))

        theta_dot = (theta - theta0) / dt
        theta_dot_norm = sqrt(inner(theta_dot, theta_dot))

        # spatial derivatives h^2|D^2()|
        grad_phi_norm = sqrt(inner(div(grad(phi)), div(grad(phi))))
        grad_theta_norm = sqrt(inner(div(grad(theta)), div(grad(theta))))

        e_D = ((R1_norm + R2_norm) * (C_h * h * h * grad_phi_norm + C_k * dt * phi_dot_norm)
              + R3_norm * (C_h * h * h * grad_theta_norm + C_k * dt * theta_dot_norm))

        # primal stabilization terms
        u_mag = sqrt(dot(u, u))
        d1 = 4.0 / sqrt((1.0 / dt)**2 + (u_mag / h)**2)
        d2 = 2.0 * h * u_mag

        e_M = abs(d1*inner((u - u0)/dt + grad(u)*u + grad(p), grad(phi)*u)
                  + d2*div(u)*div(phi))

        total_error_n = project((e_D + e_M) * dt, WDG)
        err_ind.vector()[:] += total_error_n.vector()[:]

    return err_ind


def adaptive_mesh_refinement(mesh, beta, nu):

    # error indicator parameters
    C_h = 1/8.0
    C_k = 1/2.0

    for _ in range(1):

        primal_solutions, adjoint_solutions, F_D, F_L, time = solve_NS(mesh, beta)

        # compute local error indicators
        err_ind = compute_error_indicators(primal_solutions, adjoint_solutions, mesh, dt, nu, C_h, C_k)
        err_ind_abs = np.abs(err_ind.vector())
        err_ind_mean = err_ind_abs.sum()/err_ind.vector().size()

        # local mesh refinement
        cell_marker = MeshFunction("bool", mesh, mesh.topology().dim())
        for c in cells(mesh):
          cell_marker[c] = False
          local_error_cell = err_ind_abs[c.index()]
          if local_error_cell > err_ind_mean:
              cell_marker[c] = True
        mesh = refine(mesh, cell_marker)


        plt.figure()
        plot(err_ind)
        plt.title("error indicators")
        plt.show()

        plt.figure()
        plot(mesh)
        plt.title("adapted mesh")
        plt.show()

    return mesh



# viscosity
nu_list = [rc*2*(10**(-2*n)) for n in range(5)]
print(nu_list)

# characteristic velocity
uin = 1.0

# time step length
dt = 0.1*mesh.hmin()

beta_list = [0, 2, 8, 32, 128]
#beta_list = [0, 2, 128]
#beta_list = [128]


for nu in nu_list:

    # reynolds number
    Re = uin*2*rc/nu

    drag_dict = {}
    lift_dict = {}
    C_D_dict = {}
    C_L_dict = {}
    time = None

    for beta in beta_list:
        mesh = generate_mesh(Rectangle(Point(0.0,0.0), Point(length,height)) - Circle(Point(xc,yc),rc), resolution)
        #mesh = adaptive_mesh_refinement(mesh, beta, nu)
        primal_sols, adj_sols, F_D, F_L, time = solve_NS(mesh, beta, nu)
        drag_dict[beta] = np.array(F_D)
        lift_dict[beta] = np.array(F_L)
        C_D_dict[beta] = 2.0 * drag_dict[beta] / ((uin**2) * 2*rc)
        C_L_dict[beta] = 2.0 * lift_dict[beta] / ((uin**2) * 2*rc)


    cmap = plt.get_cmap('viridis')
    colors = cmap(np.linspace(0, 1, len(beta_list)))

    # drag
    plt.figure(figsize=(6,4))
    for i, beta in enumerate(beta_list):
        c = colors[i]
        plt.plot(time,
                drag_dict[beta],
                color=c,
                label=f'beta={beta}',
                alpha=0.75)
    plt.title(f"drag, Re = {Re}")
    plt.xlabel("time")
    plt.ylabel("F_D")
    plt.legend()
    plt.tight_layout()
    plt.show()

    # lift
    plt.figure(figsize=(6,4))
    for i, beta in enumerate(beta_list):
        c = colors[i]
        plt.plot(time,
                lift_dict[beta],
                color=c,
                label=f'beta={beta}',
                alpha=0.75)
    plt.title(f"lift, Re = {Re}")
    plt.xlabel("time")
    plt.ylabel("F_L")
    plt.legend()
    plt.tight_layout()
    plt.show()

    avg_drag = [np.mean(drag_dict[beta]) for beta in beta_list]
    avg_lift = [np.mean(lift_dict[beta]) for beta in beta_list]
    var_lift = [np.var(lift_dict[beta]) for beta in beta_list]

    np.savez(f"force_stats_{Re}.npz",
            beta_list=np.array(beta_list),
            avg_lift=np.array(avg_lift),
            var_lift=np.array(var_lift),
            avg_drag=np.array(avg_drag))

    np.savez(f"force_timeseries_{Re}.npz",
            time=np.array(time),
            drag={b: drag_dict[b] for b in beta_list},
            lift={b: lift_dict[b] for b in beta_list})

    plt.figure(figsize=(6, 4))
    plt.plot(beta_list, avg_lift, label="lift")
    plt.plot(beta_list, var_lift, label="variance")
    plt.title(f"time-averaged lift, Re = {Re}")
    plt.xlabel("beta")
    plt.ylabel("force")
    plt.legend()
    plt.grid(True)
    plt.show()
